# Simulating Single Phase Transport 

The point of an OpenPNM simulation is ultimately to compute some transport process.  In this notebook we will cover the following subjects:

- Defining conductance 
- Settings boundary conditions 


Start by defining a network.  We'll use the ``Demo`` class which happens to include all the geometrical pore-scale models already. 

In [1]:
import numpy as np
import openpnm as op
op.visualization.set_mpl_style()

pn = op.network.Demo(shape=[5, 5, 1], spacing=5e-5)
print(pn)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan na

## Define Phase Viscosity

To fully illustrate the process of performing transport calculations, we'll use an empty ``Phase`` object and add all the needed properties manually:

In [2]:
water = op.phase.Phase(network=pn)

Let's assume that we are interested in pressure driven flow. This requires knowing the viscosity of the phase, so let's add a pore-scale model for computing the viscosity of water:

In [3]:
water.add_model(propname='pore.viscosity',
                model=op.models.phase.viscosity.water_correlation)
print(water)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]

══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.Phase at 0x2e0850955e0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.temperature                                                  25 / 25
  3  pore.pressure                                                     25 / 25
  4  pore.viscosity                                                    25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                           

And we can check the individual values to verify they make sense:

In [4]:
print(water['pore.viscosity'])

[0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319]


## Basic Conductance Calculation

Determining the conductance of the conduits between each pair of connected pores is the most important part of performing a simulation. The details of conductance models are covered elsewhere. For this demonstration will assume the very simplest case where all pressure loss occurs in the throats.

Recall the Hagan-Poiseuille equation for fluid flow through a cylindrical tube:

$$ Q = \frac{\pi R^4}{8 \mu L} \Delta P$$

where $R$ and $L$ are the radius and length of the throat, and $\mu$ is the viscosity of the fluid.  Together this prefactor can be referred to as the hydraulic conductance, $g_h$, giving:

$$ Q = g_h \Delta P $$

So the aim is the compute values of $g_h$ for each throat.  We start by doing this manually:

In [5]:
R = pn['throat.diameter']/2
L = pn['throat.length']
mu = water['throat.viscosity']  # See ProTip below
water['throat.hydraulic_conductance'] = np.pi*R**4/(8*mu*L)
print(water['throat.hydraulic_conductance'])

[9.20427152e-15 7.35551641e-15 1.43782360e-14 2.21592040e-14
 8.58410110e-15 1.82497731e-15 1.19043989e-15 1.35431656e-15
 1.24601001e-14 6.51791879e-15 8.29384384e-15 1.27944400e-14
 4.06680462e-15 4.35974347e-15 1.54483860e-15 1.48004403e-15
 5.12411085e-16 5.13533658e-16 7.51257701e-16 1.92035129e-15
 4.10880926e-14 7.00911817e-15 1.90538445e-15 1.39900879e-15
 1.52488506e-14 1.90114657e-14 7.77942911e-15 1.80259133e-15
 1.65935565e-15 1.01188512e-14 4.51169949e-15 4.64142945e-15
 5.11569304e-15 2.03743977e-15 2.55209825e-15 7.44866870e-16
 5.56731901e-16 7.98229431e-16 1.44963284e-15 2.88211959e-15]


```{admonition} Phase can do automatic interpolation to get throat values
  The ``Phase`` class has a special ability to interpolate pore values to throats, and vice-versa. In PNM simulations all the balances are solved for each pore, so the thermodynamic properties like temperature, pressure, etc. are all defined on pores. Consequently, the physical properties are also defined in pores, like viscosity. However, as shown above we often want viscosity values in the throats. OpenPNM provides a shortcut for this, such that if you request a throat property that does not exist, it will attempt to fetch the pores values and do a linear interpolation of values to produce an array of throat values.  There is also a function for this, ``water.interpolate_data('throat.viscosity')``, but the ``water['throat.viscosity']`` shortcut is very convenient. The automatic interpolation can be disabled in the `phase.settings`.
```

In [6]:
water.interpolate_data('throat.viscosity')

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

In [7]:
water['throat.viscosity']

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

## Create Algorithm Object

OpenPNM contains a variety of ``Algorithm`` classes in the ``openpnm.algorithms`` module.  Let's initialize a ``StokesFlow`` algorithm, since this simulates pressure driven flow through the network. 

In [8]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x2e0850baef0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.bc.rate                                                       0 / 25
  3  pore.bc.value                                                      0 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                               25
  3  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

## Assign Boundary Conditions

As can be seen in the print-out above there are predefined ``'pore.bc'`` arrays, but they contain no valid values, meaning they are all ``nans``. Once we set some boundary conditions, this will change.  Let's apply pressure BCs on one side of the network, and rate BCs on the other:

In [9]:
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x2e0850baef0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.bc.rate                                                       5 / 25
  3  pore.bc.value                                                      5 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                               25
  3  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

```{admonition} All boundary conditions are preceeded with 'pore.bc'
  All boundary conditions are stored as 'pore.bc.<type>', which means that OpenPNM's dictionary lookup tricks can be used to see all types and values of bc's using: `sf['pore.bc']` which will return a `dict`. This can be used as shown below:


In [10]:
print(sf['pore.bc'].keys())

dict_keys(['rate', 'value'])


Now we can see there are 5 valid values of each type.  The ``sf`` algorithm will look for ``'throat.hydraulic_conductance'`` on ``water`` by default, so we can just ``run``:

In [11]:
soln = sf.run()

The ``run`` method solves the mass balance around each pore and computes the pressure within each pore that is required to sustain the flow defined by the boundary conditions.  The ``soln`` object that is returned is a dictionary with the key corresponding to the quantity that was solved for.  

In [12]:
print(soln)

None


The reason for the dict format is to provide a consistent API for single components and multiphysics, where multiple different quantities might be solved for.  However, these ``'pore.pressure'`` values are also written to the dictionary of the algorithm object as well:

In [13]:
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x2e0850baef0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.bc.rate                                                       5 / 25
  3  pore.bc.value                                                      5 / 25
  4  pore.pressure                                                     25 / 25
  5  pore.initial_guess                                                25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                        

Finally we can look at how much pressure was required in the "right" pores to meet the required flow rate:

In [14]:
sf['pore.pressure'][pn.pores('right')]

array([280656.65723169, 289505.00671955, 262927.51216542, 232882.04746342,
       218303.41912755])

So we can see that 150 kPa was required to accomplish the requested flow. 

## Rigorous Conductance Calculation

The above demonstration used a very simplistic conductance calculation.  It was also stated that computing conductance is the most important part of doing a PNM simulation.  To finish this notebook, we'll look more closely at this process.

### Manual Method
Let's print the network object again:

In [15]:
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x2e0f082ed10>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.coords                                                       25 / 25
  3  throat.conns                                                      40 / 40
  4  pore.coordination_number                                          25 / 25
  5  pore.max_size                                                     25 / 25
  6  throat.spacing                                                    40 / 40
  7  pore.seed                                                         25 / 25
  8  pore.diameter                                                     25 / 25
  9  throat.max_size                                                   40 / 40
 10  

Note the ``'throat.hydraulic_size_factors'`` array.  This is computed by a pore-scale model on the ``Demo`` network.  This computation is more rigorous in the following ways:

1. The conductance of each half pore and that throat is considered.
1. The throat length is computed carefully by accounting for the 'lens' between the intersection of the spherical pore bodies and the cylindrical throat.
1. The net cross-sectional area of the pores are computed by integrating between the pore center and the pore-throat intersection point

The conductance of each element in the conduit is returned as an *Nt-by-3* array, where columns 1 and 3 contain the hydraulic conductance of the half pore on either end of the throat, and the column 1 contains the throat conductance.

In [16]:
pn['throat.hydraulic_size_factors']

array([[2.77545187e-16, 8.22117200e-18, 1.45034306e-16],
       [1.45034306e-16, 6.56988067e-18, 1.70822585e-16],
       [2.26887502e-16, 1.28425102e-17, 2.52668970e-16],
       [3.05186878e-16, 1.97924004e-17, 3.14163388e-16],
       [2.19808219e-16, 7.66724140e-18, 1.52188295e-16],
       [8.01864556e-17, 1.63005321e-18, 5.48538206e-17],
       [4.67656335e-17, 1.06329013e-18, 4.27172270e-17],
       [4.27172270e-17, 1.20966329e-18, 8.08160187e-17],
       [2.27595445e-16, 1.11292486e-17, 2.07923286e-16],
       [1.57390932e-16, 5.82174606e-18, 1.34059463e-16],
       [1.34059463e-16, 7.40798623e-18, 2.67561809e-16],
       [3.23356017e-16, 1.14278779e-17, 1.81617674e-16],
       [1.00622084e-16, 3.63243307e-18, 1.02180794e-16],
       [1.05047444e-16, 3.89408340e-18, 1.09098539e-16],
       [6.80303400e-17, 1.37983586e-18, 4.94904692e-17],
       [4.94904692e-17, 1.32196193e-18, 5.58517966e-17],
       [2.55706461e-17, 4.57680942e-19, 2.36554026e-17],
       [2.36554026e-17, 4.58683

This data is called the ``size factor`` because it is purely the geometrical information required for the computation of the hydraulic conductance.  So the Hagan-Poisseiulle equation for each element is written as:

$$ Q = \frac{F_h}{\mu} \Delta P = g_h \Delta P$$

Note that both the $\frac{\pi R^4}{8}$ term and $L$ have been rolled into the $F_h$ value.

The total conductance of the pore-throat-pore conduit can be found as the sum of three resistors in series.  Since we have conductance values, we add the inverses, and invert again.  The full expression for the hydraulic conductance between pores i and j, through throat k, is:


$$ Q = \bigg( \frac{\mu}{F_{h, i}} + \frac{\mu}{F_{h, k}} + \frac{\mu}{F_{h, j}} \bigg) ^ {-1} \Delta P $$

This can be computed by hand as follows. Note that we write the hydraulic conductance in an Nt by 2 array  the OpenPNM convention is to store conductance values in an Nt by 2 array for symmetric and asymmetric physics.

In [17]:
F_h = water['throat.hydraulic_size_factors']
gh = (mu * (1/F_h).sum(axis=1))**(-1)
gh = np.vstack((gh, gh)).T
water['throat.hydraulic_conductance'] = gh

In [18]:
water['throat.hydraulic_conductance']

array([[8.47300580e-15, 8.47300580e-15],
       [6.78704028e-15, 6.78704028e-15],
       [1.29834218e-14, 1.29834218e-14],
       [1.96472322e-14, 1.96472322e-14],
       [7.90970777e-15, 7.90970777e-15],
       [1.73799981e-15, 1.73799981e-15],
       [1.13631934e-15, 1.13631934e-15],
       [1.29812583e-15, 1.29812583e-15],
       [1.13024469e-14, 1.13024469e-14],
       [6.03278805e-15, 6.03278805e-15],
       [7.65859432e-15, 7.65859432e-15],
       [1.16496923e-14, 1.16496923e-14],
       [3.79490415e-15, 3.79490415e-15],
       [4.06403219e-15, 4.06403219e-15],
       [1.47385271e-15, 1.47385271e-15],
       [1.40905507e-15, 1.40905507e-15],
       [4.94010896e-16, 4.94010896e-16],
       [4.95183600e-16, 4.95183600e-16],
       [7.22325282e-16, 7.22325282e-16],
       [1.83235280e-15, 1.83235280e-15],
       [3.54476865e-14, 3.54476865e-14],
       [6.45647720e-15, 6.45647720e-15],
       [1.81591526e-15, 1.81591526e-15],
       [1.34100049e-15, 1.34100049e-15],
       [1.374555

In [19]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([289394.33735155, 298579.87553587, 271558.25345862, 240901.66451631,
       225756.73289535])

As can be seen the numbers are about the same as with the simple case, but should be somewhat more correct.  In fact, these above pressures are a bit higher, which is because the total conductance of the conduit is lower due to the inclusion of the pore body lengths into the total length, compared to above where only the throat length was included.

### Pore-Scale Model Method

Instead of computing the hydraulic conductance manually as done above, there is a pore-scale model available:

In [20]:
water.add_model(propname='throat.hydraulic_conductance',
                model=op.models.physics.hydraulic_conductance.generic_hydraulic)

In [21]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([289394.33735155, 298579.87553587, 271558.25345862, 240901.66451631,
       225756.73289535])

Which gives exactly the same result, without have to manually deal with the conductances-in-series calculation.